Load and clean data

In [ ]:
import sys
import time
from pathlib import Path
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV, LeaveOneGroupOut

# Set ROOT path to access other directories in project
ROOT = Path.cwd().parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

import SnowDepth.data_loader as DL
import SnowDepth.data_splitter as DS
import SnowDepth.optimal_features as OF
from SnowDepth.config import HOLDOUT_AOI
from SnowDepth.config import SEED

In [ ]:
# Assign seed
seed = SEED

# Path to TIFF files
data_dir = ROOT/"data"/"tif_files"

# Select holdout AOI
holdout_aoi = HOLDOUT_AOI

# Select max amount of features to select from FF algos
top_k = 10

# Load dataframe
df = DL.build_df(str(data_dir), drop_invalid=True, upper_threshold=3)

# Development dataframe we will use for training models
dev_df  = df[df['aoi_name'] != holdout_aoi].copy()

In [ ]:
# Run Feature filtering algorithms
ff_algos = OF.optimal_feature_sets(dev_df, top_k=10, n_per_aoi=10000)
base_cols = ["aoi_name", "row", "col", "SD"]

# HSIC
dev_df_HSIC  = dev_df[base_cols + ff_algos["HSIC"]].copy()
# PCC
dev_df_PCC  = dev_df[base_cols + ff_algos["PCC"]].copy()
# MI
dev_df_MI  = dev_df[base_cols + ff_algos["MI"]].copy()

Block HSIC Lasso B = 20.
M set to 3.
Using Gaussian kernel for the features, Gaussian kernel for the outcomes.
HSIC selected: ['IAFE', 'Gamma_VH_RTC', 'cos_Aspect', 'Gamma_VV_RTC', 'Beta_ratio', 'Slope', 'LIA', 'Gamma_RTC_ratio', 'Beta_VH', 'sin_Aspect']
PCC selected: ['IAFE', 'cos_Aspect', 'Gamma_VH_RTC', 'Gamma_VV_RTC', 'Elevation', 'Beta_VH', 'Slope', 'LIA']
MI selected): ['IAFE', 'Elevation', 'Gamma_VH_RTC', 'Gamma_VV_RTC', 'Gamma_RTC_sum', 'cos_Aspect', 'Beta_VH', 'Slope', 'Gamma_ratio']


Split data for training XGBoost

In [ ]:
# HSIC
X_dev_HSIC, y_dev_HSIC, groups_HSIC = DS.ML_split(
    dev_df=dev_df_HSIC,
    pxs_per_aoi=10000
)
# PCC
X_dev_PCC, y_dev_PCC, groups_PCC = DS.ML_split(
    dev_df=dev_df_PCC,
    pxs_per_aoi=10000
)
# MI
X_dev_MI, y_dev_MI, groups_MI = DS.ML_split(
    dev_df=dev_df_MI,
    pxs_per_aoi=10000
)

Total samples: 50000 across 5 AOIs
Features used: ['IAFE', 'Gamma_VH_RTC', 'cos_Aspect', 'Gamma_VV_RTC', 'Beta_ratio', 'Slope', 'LIA', 'Gamma_RTC_ratio', 'Beta_VH', 'sin_Aspect']
X_dev shape: (50000, 10)
X_hold shape: (1655811, 10)
Total samples: 50000 across 5 AOIs
Features used: ['IAFE', 'cos_Aspect', 'Gamma_VH_RTC', 'Gamma_VV_RTC', 'Elevation', 'Beta_VH', 'Slope', 'LIA']
X_dev shape: (50000, 8)
X_hold shape: (1655811, 8)
Total samples: 50000 across 5 AOIs
Features used: ['IAFE', 'Elevation', 'Gamma_VH_RTC', 'Gamma_VV_RTC', 'Gamma_RTC_sum', 'cos_Aspect', 'Beta_VH', 'Slope', 'Gamma_ratio']
X_dev shape: (50000, 9)
X_hold shape: (1655811, 9)


Train XGBoost and tune hyperparameters

In [ ]:
param_dist = {
    "n_estimators": [400, 600, 800, 1000],
    "learning_rate": [0.03, 0.05, 0.07, 0.1, 0.15],
    "max_depth": [4, 5, 6, 8, 10],
    "min_child_weight": [1, 2, 4, 6, 8, 12, 16],
    "subsample": [0.6, 0.7, 0.8, 0.9, 1.0],
    "colsample_bytree": [0.6, 0.7, 0.8, 0.9, 1.0],
    "reg_lambda": [0, 0.5, 1, 2, 5, 10],
    "reg_alpha": [0, 1e-4, 1e-3, 1e-2, 0.1, 1],
    "max_bin": [256, 512]
}

def run_xgb_search(tag, X, y, groups):
    xgb = XGBRegressor(
        objective="reg:squarederror",
        tree_method="hist",
        n_jobs=-1,
        random_state=SEED,
        eval_metric="rmse"
    )

    logo = LeaveOneGroupOut()
    search = RandomizedSearchCV(
        estimator=xgb,
        param_distributions=param_dist,
        n_iter=30,
        cv=logo,
        scoring="neg_root_mean_squared_error",
        n_jobs=-1,
        verbose=2,
        random_state=SEED
    )

    start = time.time()
    search.fit(X, y, groups=groups)
    end = time.time()
    elapsed = (end - start) / 60.0 

    best_rmse = -search.best_score_
    best_params = search.best_params_

    print(f"\nResults - XGBoost with {tag} feature set")
    print(f"Best hyperparameters: {best_params}")
    print(f"Best CV RMSE: {best_rmse:.4f}")
    print(f"Training time: {elapsed:.2f} minutes")

    return tag, best_rmse, best_params, elapsed


# Run across feature sets
xgb_results = {}
timings = {}

for tag, X, y, g in [
    ("HSIC", X_dev_HSIC, y_dev_HSIC, groups_HSIC),
    ("PCC",  X_dev_PCC,  y_dev_PCC,  groups_PCC),
    ("MI",   X_dev_MI,   y_dev_MI,   groups_MI),
]:
    tag, rmse, params, time_min = run_xgb_search(tag, X, y, g, seed)
    xgb_results[tag] = (rmse, params)
    timings[tag] = time_min

Fitting 5 folds for each of 30 candidates, totalling 150 fits
Results - XGBoost with HSIC feature set
HSIC — Best hyperparameters: {'subsample': 0.9, 'reg_lambda': 0.5, 'reg_alpha': 1, 'n_estimators': 600, 'min_child_weight': 2, 'max_depth': 10, 'max_bin': 512, 'learning_rate': 0.03, 'colsample_bytree': 0.7}
HSIC — Best CV RMSE: 0.5146892786026


In [ ]:
# Print leaderboard
print("\nCross-validation RMSE results (XGBoost):")
for name, (rmse, params) in xgb_results.items():
    print(f"\n{name} — CV RMSE: {rmse:.4f}")
    print(f"{name} — Best hyperparameters: {params}")
    print(f"{name} — Training time: {timings[name]:.2f} minutes")

# Find best model
best_xgb_method = min(xgb_results, key=lambda k: xgb_results[k][0])
best_xgb_rmse, best_xgb_params = xgb_results[best_xgb_method]

print(f"\n🏆 Best feature set with XGBoost: {best_xgb_method} "
      f"(CV RMSE = {best_xgb_rmse:.4f}, time = {timings[best_xgb_method]:.2f} min)")
print(f"Best XGB hyperparameters: {best_xgb_params}")


Fitting 5 folds for each of 30 candidates, totalling 150 fits
Results - XGBoost with PCC feature set
PCC — Best hyperparameters: {'subsample': 1.0, 'reg_lambda': 10, 'reg_alpha': 1, 'n_estimators': 1000, 'min_child_weight': 2, 'max_depth': 6, 'max_bin': 256, 'learning_rate': 0.1, 'colsample_bytree': 0.6}
PCC — Best CV RMSE: 0.5190581262111664
